In [42]:
import json
import pandas as pd
from pedal_defs import PEDALS  


def cc_to_percent(value: int) -> float:
    """Convert a CC value (0–127) to a percentage (0–100)."""
    try:
        return round(value * 100 / 127, 1)
    except Exception:
        return 0.0


def find_pedal_by_channel(channel: int):
    """Return (pedal_name, pedal_data) for a given MIDI channel number."""
    for name, data in PEDALS.items():
        if data.get("channel") == channel:
            return name, data
    return None, None


def interpret_mc6_preset(json_data, to_csv=False, csv_path="preset_summary.csv"):
    """Interpret Morningstar MC6 JSON preset into a readable DataFrame with structured columns."""
    # --- Load JSON ---
    if isinstance(json_data, str):
        try:
            if json_data.strip().endswith(".json"):
                with open(json_data, "r") as f:
                    preset = json.load(f)
            else:
                preset = json.loads(json_data)
        except Exception as e:
            raise ValueError(f"Error loading JSON: {e}")
    else:
        preset = json_data

    # --- Context ---
    bank = preset.get("bankNum", "N/A")
    preset_name = preset.get("shortName") or preset.get("longName") or "(Unnamed Preset)"

    rows = []

    # --- Parse each message ---
    for msg in preset.get("msgArray", []):
        m = msg.get("m")
        t = msg.get("t")
        c = msg.get("c")
        data = msg.get("data", [])
        a = msg.get("a", 0)

        if not a and all(v == 0 for v in data):
            continue

        pedal_name, pedal_info = find_pedal_by_channel(c)
        msg_type = "Unknown"
        parameter = ""
        value = ""
        desc = ""

        if t == 7:  # Clock
            msg_type = "Clock"
            bpm = data[1] if len(data) > 1 else "?"
            desc = f"Start MIDI clock at {bpm} BPM"
            value = bpm

        elif t == 1:  # Program Change
            msg_type = "Program Change"
            program = data[0] if data else 0
            parameter = "Preset"
            value = program
            desc = f"Preset {program} on {pedal_name}" if pedal_name else f"Program Change {program} on Channel {c}"

        elif t == 2:  # Control Change
            msg_type = "Control Change"
            if len(data) >= 2:
                cc_num, cc_val = data[:2]
                percent = cc_to_percent(cc_val)
                value = f"{percent}%"
                if pedal_info:
                    parameter = pedal_info["cc_map"].get(cc_num, f"CC {cc_num}")
                    desc = f"{parameter} = {percent}% on {pedal_name}"
                else:
                    parameter = f"CC {cc_num}"
                    desc = f"{parameter} = {percent}% on Channel {c}"
            else:
                desc = f"Incomplete CC message on Channel {c}"

        rows.append({
            "msgIndex": m,
            "Type": msg_type,
            "Channel": c,
            "Pedal": "" if msg_type == "Clock" else (pedal_name or ""),
            "Parameter": parameter,
            "Value": value,
        })

    # --- Create DataFrame ---
    df = pd.DataFrame(rows).sort_values("msgIndex", na_position="last").reset_index(drop=True)

    # Add preset label row for readability
    header = pd.DataFrame({
        "msgIndex": [""],
        "Type": [""],
        "Channel": [""],
        "Pedal": [""],
        "Parameter": [""],
        "Value": [""],
    })
    df = pd.concat([header, df], ignore_index=True)

    if to_csv:
        df.to_csv(csv_path, index=False)
        print(f"Saved CSV to {csv_path}")

    return df


if __name__ == "__main__":
    df = interpret_mc6_preset("preset_A_data(2).json", to_csv=True)
    print(df)



Saved CSV to preset_summary.csv
  msgIndex            Type Channel      Pedal   Parameter  Value
0                                                               
1        0           Clock       1                            81
2        1  Program Change       2     Preamp      Preset     12
3        2  Control Change       1    Big Sky         Mix  46.5%
4        3  Control Change       1    Big Sky       Decay  52.0%
5        4  Control Change       3    Volante  Echo Level   0.0%
6        5  Control Change       3    Volante     Repeats  54.3%
7        6  Program Change       4  Microcosm      Preset     45
8        7  Program Change       7       Deco      Preset      0
